In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
key = os.getenv('AZURE_SEARCH_KEY')

In [ ]:
from fastbook import search_images_bing
from PIL import Image
from fastdownload import download_url

res = search_images_bing(key, 'grizzly bear')
images = res.attrgot('contentUrl')
dest = './images/grizzly.jpg'
download_url(images[0], dest)

im = Image.open(dest)
im.to_thumb(128,128)

In [2]:
from fastai.vision.utils import download_images
from pathlib import Path

bears_types = ('grizzly', 'black', 'teddy')
path = Path('bears')

이미지 다운로드

In [ ]:
if not path.exists():
    path.mkdir()
    for o in bears_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        res = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=res.attrgot('contentUrl'))

In [3]:
from fastai.vision.utils import verify_images
from fastai.data.transforms import get_image_files

fns = get_image_files(path) # get all files in path
failed = verify_images(fns) # verify all files in path
failed.map(Path.unlink) # delete all failed files

(#0) []

In [4]:
from fastai.vision.augment import Resize, __all__
from fastai.data.transforms import RandomSplitter, parent_label
from fastai.vision.data import ImageBlock
from fastai.data.block import DataBlock, CategoryBlock
from fastai.data.transforms import get_image_files

bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), # 1번째는 독립변수 (입력 데이터), 2번째는 종속변수 (독립변수를 활용해 예측할 대상)
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label, # 독립변수는 x, 종속변수는 y
    item_tfms=Resize(224)
)

data_loaders = bears.dataloaders(path)

In [ ]:
data_loaders.show_batch(max_n=4, nrows=1)

사진 이미지가 다를 경우 학습에 불리 할 수 있다. 그래서 이미지를 표준화 시켜준다. (Squish = 찌그러트림)

In [ ]:
from fastai.vision.all import *

bears = bears.new(item_tfms=Resize(128, ResizeMethod.Squish))
data_loaders = bears.dataloaders(path)
data_loaders.show_batch(max_n=9, nrows=3)

랜덤으로 각 epoch 마다 같은 이미지 속 다양한 부분을 학습 시킴.

In [ ]:
from fastai.vision.all import *

bears = bears.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
data_loaders = bears.dataloaders(path)
data_loaders.show_batch(max_n=4, nrows=1, unique=True)

데이터 증강: 입력 데이터를 임의로 변경해 새로운 데이터를 생성하는 기법.

In [ ]:
from fastai.vision.all import *

bears = bears.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=4))
data_loaders = bears.dataloaders(path)
data_loaders.train.show_batch(max_n=8, nrows=2, unique=True)

모델 훈련과 훈련된 모델을 이용한 데이터 정리

In [5]:
from fastai.vision.all import *

bears = bears.new(item_tfms=RandomResizedCrop(224, min_scale=0.5), batch_tfms=aug_transforms())
data_loaders = bears.dataloaders(path)

learn = vision_learner(data_loaders, resnet18, metrics=error_rate)
learn.fine_tune(4)

Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


NotImplementedError: The operator 'aten::_linalg_solve_ex.result' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

학습 완료 후 모델 오차 행렬 파악

In [6]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

NameError: name 'learn' is not defined